In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv3D, Conv2D, Conv1D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

Using TensorFlow backend.


In [2]:
from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance

# Image Generator

In [3]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input

image_size = 128

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

data_generator_no_aug = ImageDataGenerator(
                                    preprocessing_function=preprocess_input
                                    )

train_generator = data_generator_with_aug.flow_from_directory(
        'datasets/train/',
        target_size=(image_size, image_size),
        batch_size=200,
        class_mode='categorical')

data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = data_generator_no_aug.flow_from_directory(
        'datasets/validation/',
        target_size=(image_size, image_size),
        batch_size=200,
        class_mode='categorical')

Found 11973 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Model

In [4]:
model = Sequential()

model.add(Conv2D(filters = 30, kernel_size = (5, 5),padding = 'Same', 
                 activation ='relu', input_shape = (128,128,3), strides = 2))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(MaxPool2D(pool_size=(2, 2), strides = 2))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)
model.add(Dropout(0.25))

model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation = 'relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(MaxPool2D(pool_size=(2, 2), strides = 2))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)
model.add(Dropout(0.25))

model.add(Conv2D(filters = 16, kernel_size = (3, 3), padding = 'Same', 
                activation = 'relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Conv2D(filters = 16, kernel_size = (3, 3), padding = 'Same', 
                activation = 'relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Conv2D(filters = 16, kernel_size = (3, 3), padding = 'Same', 
                activation = 'relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Conv2D(filters = 16, kernel_size = (3, 3), padding = 'Same', 
                activation = 'relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))

Input:  (None, 128, 128, 3)
Output:  (None, 64, 64, 30)
Input:  (None, 128, 128, 3)
Output:  (None, 32, 32, 30)
Input:  (None, 128, 128, 3)
Output:  (None, 32, 32, 16)
Input:  (None, 128, 128, 3)
Output:  (None, 32, 32, 16)
Input:  (None, 128, 128, 3)
Output:  (None, 32, 32, 16)
Input:  (None, 128, 128, 3)
Output:  (None, 16, 16, 16)
Input:  (None, 128, 128, 3)
Output:  (None, 16, 16, 16)
Input:  (None, 128, 128, 3)
Output:  (None, 16, 16, 16)
Input:  (None, 128, 128, 3)
Output:  (None, 16, 16, 16)
Input:  (None, 128, 128, 3)
Output:  (None, 16, 16, 16)


In [5]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)


In [6]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


In [7]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)


In [8]:
epochs = 500
batch_size = 100

In [9]:
# Fit Model Here
history = model.fit_generator(
        train_generator,
        steps_per_epoch=23,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/500
 5/23 [=====>........................] - ETA: 1:32 - loss: 2.1528 - acc: 0.4810

/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))
/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag))


18/23 [======================>.......] - ETA: 17s - loss: 1.0967 - acc: 0.5350

/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))
/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag))


23/23 [==============================] - 74s 3s/step - loss: 1.0053 - acc: 0.5485 - val_loss: 0.6990 - val_acc: 0.4960
Epoch 2/500
23/23 [==============================] - 58s 3s/step - loss: 0.6809 - acc: 0.5803 - val_loss: 0.7153 - val_acc: 0.5000
Epoch 3/500
14/23 [=================>............] - ETA: 22s - loss: 0.6775 - acc: 0.5918

/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))
/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag))
/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))
/home/holy/anaconda3/envs/cv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag))


23/23 [==============================] - 44s 2s/step - loss: 0.6715 - acc: 0.5963 - val_loss: 0.6994 - val_acc: 0.4930
Epoch 4/500
16/23 [===================>..........] - ETA: 6s - loss: 0.6667 - acc: 0.5903

KeyboardInterrupt: 

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(2)) 
